# Tensorflow로 다중 선형 회귀 모델 만들고 실행하기


In [1]:
#import, 함수 선언 부
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#학습할 데이터 각 행의 최대, 최소값을 저장하는 array
minArr = []
maxArr = []

#Extracurricular Activities 행의 데이터를 이진화 #YES->1 / NO->0
def BinaryAct(df):
    df = df.replace({"Extracurricular Activities":"Yes"}, 1)
    df = df.replace({"Extracurricular Activities":"No"}, 0)

    return df

def Normalization(df):
    for i in range(5):
        tmp = df[df.columns[i]]
        df[df.columns[i]] =  (tmp - tmp.min()) / (tmp.max() - tmp.min())

        minArr.append(tmp.min())
        maxArr.append(tmp.max())

    return df

def NormNewData(df):
    if minArr and maxArr is None:
        print("Normalize origin data First!!\n")
        return
    
    for i in range(5):
        tmp = df[df.columns[i]]
        df[df.columns[i]] =  (tmp - minArr[i]) / (maxArr[i] - minArr[i])
    
    return df

In [2]:
#csv파일 읽어서 dataframe 객체 생성, 이진화, normalize
df = pd.read_csv("./data/StudentPerformance.csv", encoding="cp949")
df = BinaryAct(df)
df = Normalization(df)
print(df.head())

#df slicing 통해 x, y 분리
x = df.loc[:, "Hours Studied":"Sample Question Papers Practiced"]
y = df.loc[:, "Performance Index"]
print(x.head())
print(y.head())

   Hours Studied  Previous Scores  Extracurricular Activities  Sleep Hours   
0          0.750         1.000000                         1.0          1.0  \
1          0.375         0.711864                         0.0          0.0   
2          0.875         0.186441                         1.0          0.6   
3          0.500         0.203390                         1.0          0.2   
4          0.750         0.593220                         0.0          0.8   

   Sample Question Papers Practiced  Performance Index  
0                          0.111111               91.0  
1                          0.222222               65.0  
2                          0.222222               45.0  
3                          0.222222               36.0  
4                          0.555556               66.0  
   Hours Studied  Previous Scores  Extracurricular Activities  Sleep Hours   
0          0.750         1.000000                         1.0          1.0  \
1          0.375         0.711864

In [3]:
#새로운 데이터 dataframe으로 생성, 이진화, normalize
#예측은 학습이후 진행함
new_data = pd.DataFrame([[8, 88, "Yes", 8, 3], [7, 93, "No", 9, 4]], 
                        columns=["Hours Studied", "Previous Scores",
                                  "Extracurricular Activities", "Sleep Hours", 
                                  "Sample Question Papers Practiced"])
new_data = BinaryAct(new_data)
new_data = NormNewData(new_data)
print(new_data.head())

   Hours Studied  Previous Scores  Extracurricular Activities  Sleep Hours   
0          0.875         0.813559                         1.0          0.8  \
1          0.750         0.898305                         0.0          1.0   

   Sample Question Papers Practiced  
0                          0.333333  
1                          0.444444  


In [4]:
#seauential 모델 생성, 선형회귀모델 적용, 학습
#선형경사하강법 sgd, 평균제곱오차 mse 적용

model = keras.Sequential()
model.add(layers.Dense(1, input_dim = 5, activation="linear"))
model.compile(optimizer="sgd", loss="mse")
model.fit(x, y, epochs=100)
#epochs는 2000번까지 돌려봤는데 loss값이 4.1 밑으로 내려가지 않아서 100까지만 설정함

Epoch 1/100
313/313 [==============================] - 2s 3ms/step - loss: 268.2780
Epoch 2/100
313/313 [==============================] - 1s 3ms/step - loss: 52.6748
Epoch 3/100
313/313 [==============================] - 1s 2ms/step - loss: 22.3612
Epoch 4/100
313/313 [==============================] - 1s 3ms/step - loss: 11.6672
Epoch 5/100
313/313 [==============================] - 1s 4ms/step - loss: 7.5201
Epoch 6/100
313/313 [==============================] - 1s 4ms/step - loss: 5.7688
Epoch 7/100
313/313 [==============================] - 1s 3ms/step - loss: 4.9720
Epoch 8/100
313/313 [==============================] - 1s 3ms/step - loss: 4.5851
Epoch 9/100
313/313 [==============================] - 1s 3ms/step - loss: 4.3846
Epoch 10/100
313/313 [==============================] - 1s 4ms/step - loss: 4.2792
Epoch 11/100
313/313 [==============================] - 1s 3ms/step - loss: 4.2229
Epoch 12/100
313/313 [==============================] - 1s 3ms/step - loss: 4.1941
Epoch 13

In [5]:
#new_data predict
new_pred = model.predict(new_data)
print(f"new_data 예측 점수 : {new_pred}")

1/1 [==============================] - 0s 236ms/step
new_data 예측 점수 : [[83.35695]
 [85.68751]]


In [7]:
#기존(실제)점수와 비교
y_pred = model.predict(x)
print(f"실제 점수 : {y}")
print(f"예측 점수 : {y_pred}")

313/313 [==============================] - 0s 1ms/step
실제 점수 : 0       91.0
1       65.0
2       45.0
3       36.0
4       66.0
        ... 
9995    23.0
9996    58.0
9997    74.0
9998    95.0
9999    64.0
Name: Performance Index, Length: 10000, dtype: float64
예측 점수 : [[91.80183 ]
 [63.140068]
 [44.9998  ]
 ...
 [72.9453  ]
 [94.316925]
 [65.277176]]
